In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random

In [ ]:
data_path = "../input/aerial-cactus-identification/"

In [ ]:
test_folder = data_path + "test.zip"
train_folder = data_path + "train.zip"

In [ ]:
import os
import zipfile
unziped_data = '/kaggle/working/train'
os.mkdir(unziped_data)
with zipfile.ZipFile(train_folder,"r") as z:
    z.extractall(unziped_data)

In [ ]:
train_names = pd.read_csv(data_path + "train.csv")
train_names

# Preprocessing


In [ ]:
IMG_SIZE = 248
EPOCHS = 3
BATCH_SIZE = 32

In [ ]:
# take in an image filename and label and return a 3d numpy array of size IMG_SIZE with all the rgb values scaled from 0 to 1 and the label
def load_img(fname, label):
    img = cv2.imread("/kaggle/working/train/train/" + fname)
    img_pil = Image.fromarray(img)
    arr = np.array(img_pil.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS))
    return arr, np.array(label)

In [ ]:
# load a number of images with the load_img file
def load_batch(arr):
  # each item in array will be [2d array, label]
    img_array = []
    labels = []
    for i in range(len(arr)-1):
        temp = load_img(train_names["id"][i], train_names['has_cactus'][i])
        img_array.append(temp[0])
        if temp[1] == 0:
            labels.append([1,0])
        else:
            labels.append([0,1])
    return np.array(img_array), np.array(labels) 

In [ ]:
def batch(arr, n=32):
    l = len(arr)
    for ndx in range(0, l, n):
        yield arr[ndx:min(ndx + n, l)]

In [ ]:
class ANN():
    def __init__(self, n, m):
        self.name = n
        self.model = m

    def train_model(self, paths, epochs=EPOCHS, batch_size=BATCH_SIZE):
        for i in range(epochs):
            random.shuffle(paths)

            batches = batch(paths, batch_size)
      
            print("Epoch: " + str(i+1))
            print()
            for j, b in enumerate(batches):
                print("\nBatch: " + str(j+1))

                x, y = load_batch(b)

                self.model.fit(x, y, verbose=1)

                #no data leaks

                del x, y
                print()

    def eval(self, paths):
        x, y = load_batch(paths)

        preds = self.model.predict(x)

        return preds, y    
        print()

    def eval_model(self, paths):
        x, y = load_batch(paths)
        preds = self.model.predict(x)
        for index, pred in enumerate(preds):
            for i, item in enumerate(pred):
                if item < .5:
                    preds[index][i] = [1, 0]
                else:
                    preds[index][i] = [0, 1]
        sum = 0
        for i in range(len(y)):
            if preds[i][0] == y[i][0] and preds[i][1] == y[i][1]:
                sum += 1
        return sum, len(y)

    def save_model(self, path):
        model.save("../content/drive/My Drive/Kaggle Team/2021/Face Tracking with Masks/our_models/" + this.name + ".h5")

In [ ]:
input_t = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
xception = tf.keras.applications.Xception(include_top=False, weights="imagenet", input_tensor=input_t,
    pooling=None,
    classes=2,
    classifier_activation="softmax",
)

for layer in xception.layers[:102]: layer.trainable = False


xcept = ANN("xception", tf.keras.models.Sequential())

xcept.model.add(xception)
xcept.model.add(tf.keras.layers.Flatten())
xcept.model.add(tf.keras.layers.Dense(2, activation="softmax"))
xcept.model.summary()

In [ ]:
for i, l in enumerate(xception.layers):
    print(str(i) + " " + str(l))

In [ ]:
xcept.model.compile(loss='categorical_crossentropy', optimizer="Adam")

In [ ]:
train_paths = train_names.to_numpy()

In [ ]:
xcept.train_model(train_paths[:16000], epochs=1)

In [ ]:
preds, y = xcept.eval(train_names[16000:])

In [ ]:
for i in range(len(preds)):
    for j in range(2):
        if preds[i][j] < .5:
            preds[i][j] = 0
        else:
            preds[i][j] = 1

In [ ]:
sum = 0
for i in range(len(preds)):
    if preds[i][0] == y[i][0] and preds[i][1] == y[i][1]:
        sum += 1

In [ ]:
sum, len(preds)

In [ ]:
accuracy = sum/len(preds)
sum/len(preds), accuracy